In [1]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import matplotlib as mpl 

mpl.rcParams['figure.dpi']= 200
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

from rpy2.robjects import pandas2ri
pandas2ri.activate()

%reload_ext rpy2.ipython

In [2]:
%%R

library('lme4')
library('margins')
library("performance")
library('tidyverse')
library('broom.mixed')

getICCs <- function(m, type = 'poisson'){
  var_s <- as.numeric(getME(m, "theta")[1]^2) # seller level variance
  var_w <- as.numeric(getME(m, "theta")[2]^2) # week level variance
  
  if(type == 'poisson'){
    lambda = .139
    alpha <- log(1 + 1/lambda)
  }
  if(type == 'binomial'){
    alpha <- (pi^2) / 3
  }
  
  icc <- list(
    s = (var_s)/ (var_s + var_w + alpha),
    w = (var_w)/ (var_s + var_w + alpha),
    t = (var_s + var_w)/ (var_s + var_w + alpha),
    a = (alpha)/ (var_s + var_w + alpha)
  )
  
  return(icc)
}

sessionInfo()

R[write to console]: Loading required package: Matrix

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

R[write to console]: ✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ tidyr::pack()   masks Matrix::pack()
✖ tidyr::unpack() masks Matrix::unpack()

R[write to console]: Registered S3 method overwritten by 'broom.mixed':
  method      from 
  tidy.gamlss broom



R version 4.0.3 (2020-10-10)
Platform: x86_64-apple-darwin17.0 (64-bit)
Running under: macOS  11.2.1

Matrix products: default
LAPACK: /Library/Frameworks/R.framework/Versions/4.0/Resources/lib/libRlapack.dylib

locale:
[1] C/UTF-8/C/C/C/C

attached base packages:
[1] tools     stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] broom.mixed_0.2.6 forcats_0.5.0     stringr_1.4.0     dplyr_1.0.2      
 [5] purrr_0.3.4       readr_1.4.0       tidyr_1.1.2       tibble_3.0.4     
 [9] ggplot2_3.3.2     tidyverse_1.3.0   performance_0.5.1 margins_0.3.23   
[13] lme4_1.1-25       Matrix_1.2-18    

loaded via a namespace (and not attached):
 [1] statmod_1.4.35    TMB_1.7.19        tidyselect_1.1.0  reshape2_1.4.4   
 [5] splines_4.0.3     haven_2.3.1       lattice_0.20-41   colorspace_1.4-1 
 [9] vctrs_0.3.4       generics_0.0.2    blob_1.2.1        rlang_0.4.8      
[13] nloptr_1.2.2.2    pillar_1.4.6      withr_2.3.0       glue_1.4.2     

In [3]:
MAINDIR = os.getcwd().rsplit('/', 1)[0]
PATH = os.path.join(MAINDIR, 'data/analysis')

df = pd.read_pickle(os.path.join(PATH, 'vendor_week.pickle'))
df = df.merge(df.groupby('vendor')['me_min'].min().rename('me'), on = 'vendor')
df = df.rename(columns= {
    'international_shipment_count_w': 'int_shipment_count_w', 
    'international_shipment': 'int_shipment'})

In [4]:
from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer()

# normalize variables
skewed_variables = [
    'neg_count_min', 'pos_count_min', 'neg_count_w', 
    'neg_count_w_shift', 'pos_count_w', 'pos_count_w_shift', 'sales_volume_w', 
    'item_count_w', 'int_shipment_count_w']
bc_vars = pd.DataFrame(
    pt.fit_transform(df[skewed_variables]),
    columns = ['bc_' + var for var in skewed_variables])

c_vars = df[skewed_variables].add_prefix('c_')

dummy_vars = df.assign(
        items   = pd.qcut(df['item_count'], 3,  labels=["low", "medium", "high"]),
        items_w = pd.qcut(df['item_count_w'], 3,  labels=["low", "medium", "high"]),
        sales   = pd.qcut(df['cum_count'], 2,  labels=["low", "high"]),
        sales_w = pd.qcut(df['sales_volume_w'], 2,  labels=["low", "high"]))\
    .loc[:,['items', 'items_w', 'sales', 'sales_w']]

n_vars = df[['cum_count', 'sales_volume_w', 'item_count', 
             'item_count_w', 'me', 'int_shipment_count_w',
             'empty_stock_last_week_count_w', 'empty_stock_last_week_maxw']].rename({'cum_count': 'sales_volume'})

bool_vars = df[
    ['arf', 'arm_maxw', 'arm_maxw_shift',
     'int_shipment', 'has_price_drop', 'has_price_drop_shift']].astype('int')

lev_vars =  df[['vendor','w', 'me_min']]

data = pd.concat([lev_vars, bool_vars, bc_vars, c_vars, dummy_vars, n_vars], axis = 1)

In [5]:
%%R -i data

data['me_'] <- scale(data$me)[,1]
data['w_'] <- scale(data$w)[,1]
data['me_2'] <- scale(data$me^2)[,1]
data['w_2'] <- scale(data$w^2)[,1]

variables = c(
    'bc_pos_count_w', 'bc_item_count_w', 'bc_int_shipment_count_w',
    'bc_neg_count_w', 'c_neg_count_w', 'c_pos_count_w')

for (var in variables){
  for (vendor in unique(data$vendor)){
    # calculate mean and deviances
    x = data[data$vendor == vendor, var]
    m = mean(x)
    dev = x - m
    
    # concat information to dataframe
    data[data$vendor == vendor, paste(var, "m", sep=".")] <- m
    data[data$vendor == vendor, paste(var, "dev", sep=".")] <- dev
  }
}

In [6]:
%%R

# Create empty list for results
model1 <- list()

# Baseline Model
model1 <- append(model1, 
                 list(a = glm(c_neg_count_w_shift ~ 1,
                              data, 
                              family = poisson)))

# Fixed effects cross classified model for (vendor/week) using
# two random intercepts on using arm at a time point (shifted) 
# Including control variables
model1 <- append(model1, 
                 list(c = glmer(c_neg_count_w_shift ~ 1
                                + (1 | vendor) + (1 | w),
                                data, 
                                family = poisson)))

Optimizer <- glmerControl(optimizer = "bobyqa",
                          optCtrl = list(maxfun=2e5))

# Fixed effects cross classified model for (vendor/week) using
# two random intercepts on using arm at a time point (shifted)
# cross-level-interaction of ARF and Negative Feedbacks with 
# bobyqa optimization
model1 <- append(model1, 
                 list(e = glmer(c_neg_count_w_shift ~ arf
                                + bc_pos_count_w.m
                                + bc_pos_count_w.dev
                                + bc_item_count_w.m
                                + bc_item_count_w.dev
                                + bc_int_shipment_count_w.m
                                + bc_int_shipment_count_w.dev
                                + me_ + me_2 
                                + w_ + w_2 
                                + (1 | vendor) + (1 | w),
                                data, 
                                family = poisson,
                                control = Optimizer)))

In [72]:
%%R

r2.model1.vendor <- ((VarCorr(model1$c)$vendor - VarCorr(model1$e)$vendor) / VarCorr(model1$c)$vendor)[1,1]
print(r2.model1.vendor)

r2.model1.w <-((VarCorr(model1$c)$w - VarCorr(model1$e)$w) / VarCorr(model1$c)$w)[1,1]
print(r2.model1.w)

[1] 0.3475356
[1] 0.8800426


In [73]:
%%R

# Create empty list for results
model2 <- list()

# Baseline Model
model2 <- append(model2, 
                 list(a = glm(arm_maxw_shift ~ 1,
                              data, 
                              family = binomial)))


# Fixed effects cross classified model for (vendor/week) using
# two random intercepts on using arm at a time point (shifted) 
# Including control variables
model2 <- append(model2, 
                 list(c = glmer(arm_maxw_shift ~ 1
                                + (1 | vendor) + (1 | w),
                                data, 
                                family = binomial)))

icc2 <- getICCs(model2$c, 'binomial')


# Fixed effects cross classified model for (vendor/week) using
# two random intercepts on using arm at a time point (shifted)
# cross-level-interaction of ARF and Negative Feedbacks with 
# bobyqa optimization
model2 <- append(model2, 
                 list(e = glmer(arm_maxw_shift ~ arf
                                + bc_neg_count_w.m
                                + bc_neg_count_w.dev
                                + bc_neg_count_w.m:arf
                                + bc_neg_count_w.dev:arf
                                + bc_pos_count_w.m
                                + bc_pos_count_w.dev
                                + bc_item_count_w.m
                                + bc_item_count_w.dev
                                + bc_int_shipment_count_w.m
                                + bc_int_shipment_count_w.dev
                                + me_ + me_2 
                                + w_ + w_2 
                                + (1 | vendor) + (1 | w),
                                data, 
                                family = binomial, 
                                control = Optimizer)))

In [74]:
%%R

r2.model2.vendor <- ((VarCorr(model2$c)$vendor - VarCorr(model2$e)$vendor) / VarCorr(model2$c)$vendor)[1,1]
print(r2.model2.vendor)

r2.model2.w <-((VarCorr(model2$c)$w - VarCorr(model2$e)$w) / VarCorr(model2$c)$w)[1,1]
print(r2.model2.w)

[1] 0.8585003
[1] 0.7735609
